In [9]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import  sys


In [2]:
driver = webdriver.Chrome('C:/Users/Pc4y/Downloads/chromedriver.exe')

<ipython-input-2-7894f6f16619>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Pc4y/Downloads/chromedriver.exe')


In [10]:
# Function that parser through page and returns bs content
def parse_page(url):
    try:
        driver.get(url)
        
        content = driver.page_source
        soup = bs(content,'html.parser')
        return soup
    except:
        pass

In [11]:
# Returns a list of all models of one brand
def get_models(brand):
	brand = (brand.replace(' ', '-')).lower()
	url ='https://www.polovniautomobili.com/auto-oglasi/pretraga?brand=' + brand
	soup = parse_page(url)
	models= soup.find(id='model')
	models_list=[]
	for model in models:
		if model.get_text()=='Ostalo':
			pass
		else:
			models_list.append(model.get_text())
	return models_list[1:]

In [12]:
def replace(string):
	string = string.replace('š','s')
	string = string.replace('Š','s')
	string = string.replace('ć','c')
	string = string.replace('Ć','c')
	string = string.replace('Ž','z')
	string = string.replace('ž','z')
	string = string.replace('č','c')
	string = string.replace('Č','c')
	return string


In [13]:
def get_num_of_cars(url):
    soup = parse_page(url)
    
    if soup!=None:
        try:
            num = soup.find_all('small')
            num = num[-2].get_text()
            num = num.split(' ')[-1]
            
            return (round(int(num)/25)+1)
        except:
            return -1
        

In [14]:
# Get all cars from one brand (iterate through each model in given brand)
def get_cars(): #model

    #brand = (brand.replace(' ', '-')).lower()
    #model = (model.replace(' ', '-')).lower()
    url1 ='https://www.polovniautomobili.com/auto-oglasi/pretraga?page='
    url2 = '&sort=basic&price_from=1000&price_to=20000&year_from=2000&year_to=2020&chassis%5B0%5D=277&chassis%5B1%5D=2631&chassis%5B2%5D=278&chassis%5B3%5D=2636&chassis%5B4%5D=2632&city_distance=0&showOldNew=all&without_price=1&engine_volume_to=3000&mileage_from=50000&wheel_side=2630'
    list_of_links = []

    # find number of pages in a given model
    num = get_num_of_cars(url1+str(1)+url2)
    print(num)
    
    if num==0: 
        num=1
    # if ther are car ads on site and number of cars is greater or equal to 100
    if num!=-1:
        carList = []
        # loop over number of pages
        for i in range(1,num+1):
            #print('On page {}'.format(str(i)))
            if i%50==0:
                time.sleep(25)

            soup = parse_page(url1+str(i)+url2)
            
            if soup!=None:
                #print('Found articles')
                try:
                    for a in soup.find_all(class_="ga-title", href=True):
                        list_of_links.append(a['href'])
                        

                except:
                    print('Change here')
                    pass
            
                   
        return list_of_links         
    else:
        pass


In [15]:
# function that inserts cars into dataframe and then loads into .csv file
def insertAll():
    
    # iterate through every brand and every model of that brand and load data into dataframe
    links = []
    duplicate_models = ['PASSAT','GOLF','CORSA','ASTRA', 'VECTRA', 'SERIJA X', 'SERIJA M','SERIJA 1',
                        'SERIJA 2','SERIJA 3','SERIJA 4','SERIJA 5','SERIJA 7','SERIJA 6','SERIJA 8','SERIJA I','QUATTRO', 'R-X8']
    try:
        cars = get_cars()
        if cars!=None:
            links.append(cars)
                    
                               
        return links
    
    except Exception as e:
        print(e)
        textfile = open("all.txt", "w")
        for element in links:
            if element!=None:
                for url in element:
                    textfile.write(url + "\n")
        textfile.close()
        pass
    

In [71]:
#carList=[]
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_pages(urls):
    data = pd.DataFrame()
    carList=[]
    driver = webdriver.Chrome('C:/Users/Pc4y/Downloads/chromedriver.exe')
    driver.set_page_load_timeout(8)
    #print('Driver')
    count = 1
    
    for url in urls:
        if url!=None:

            try:
                if count%50==0:
                    time.sleep(15)
                driver.get('https://www.polovniautomobili.com'+url)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "infoBox")))
                content = driver.page_source
                soup = bs(content,'html.parser')
                #print(soup)
                

                try:
                    cena = soup.find(class_="priceClassified regularPriceColor")
                    cena_popust = soup.find("priceClassified discountedPriceColor")
                    if cena.get_text()!=None:
                        cena=cena.get_text()[:-2]
                        cena = int(cena.split('.')[0]+cena.split('.')[1])
                        cena_dict = {'Cena': cena}
                    elif cena_popust.get_text()!=None:
                        cena=cena_popust.get_text()
                        cena = int(cena.split('.')[0]+cena.split('.')[1])
                        cena_dict = {'Cena': cena}

                    opste = soup.find_all(class_="infoBox")
                    labels = []
                    info = []
                    i = 1
                    for kar in opste:
                        naziv = kar.find_all(class_="uk-width-1-2")
                        #print('*'*20)

                        if i == 1:
                            labels = [label.get_text() for label in naziv[::2]]
                            info = [info_.get_text() for info_ in naziv[1::2]]
                            info_dict = dict(zip(labels, info))
                            labels = []
                            info = []
                            i +=1
                            #break
                            
                        
                        elif i==2:
                            labels = [label.get_text() for label in naziv[::2]]
                            info = [info_.get_text() for info_ in naziv[1::2]]
                            oprema_dict = dict(zip(labels, info))
                            #print(oprema_dict)
                            labels = []
                            info = []
                            break
                            
                        
                    dict_car = {**info_dict,**cena_dict, **oprema_dict} # **oprema_dict, **dodatno, 
                    carList.append(dict_car)
                    
                except:
                    #print("Unexpected error:", sys.exc_info())
                    #data = data.append(carList,ignore_index=True)
                    #print(count)
                    pass
            except TimeoutException:
                #print("Unexpected error, driver refreshed:", sys.exc_info())
                #driver.refresh()
                #print(count)
                pass
        count+=1
        
    data = data.append(carList,ignore_index=True)
    driver.close()
    return data
                    
        

In [17]:
from selenium.webdriver.support.ui import WebDriverWait
#driver = webdriver.Chrome('C:/Users/Pc4y/Downloads/chromedriver.exe')
import time 

#list_of_links = insertAll()
"""textfile = open("all.txt", "w")
for element in list_of_links:
    if element!=None:
        for url in element:
            textfile.write(url + "\n")
textfile.close()"""
#print(len(lista))https://www.polovniautomobili.com/auto-oglasi/pretraga?brand=alfa-romeo&brand2=&price_from=1000&price_to=20000&year_from=2000&year_to=2020&chassis%5B%5D=277&chassis%5B%5D=2631&chassis%5B%5D=278&chassis%5B%5D=2632&flywheel=&atest=&door_num=&submit_1=&without_price=1&date_limit=&showOldNew=old&modeltxt=&engine_volume_from=&engine_volume_to=&power_from=&power_to=&mileage_from=&mileage_to=&emission_class=&seat_num=&wheel_side=&registration=&country=&country_origin=&city=&registration_price=&sort=&page=8

'textfile = open("all.txt", "w")\nfor element in list_of_links:\n    if element!=None:\n        for url in element:\n            textfile.write(url + "\n")\ntextfile.close()'

In [18]:
list_of_links

NameError: name 'list_of_links' is not defined

In [19]:
with open('all.txt', 'r') as f:
    lista = f.readlines()

#lista_deo = lista[:3000]
#lista_deo2 = lista[3000:6000]
#lista_deo3 = lista[6000:7000]
#lista_deo5 = lista[12000:15870]
#print(len(lista))

#data = scrape_pages(list_of_links[0][:2])

In [20]:
list_urls = [k.split('\n')[0] for k in lista]

In [66]:
len(list_urls)

39769

In [77]:
"""part = 20
for i in range(38000, 38000, 2000):
    print(i,i+2000)
    print(part)
    data = scrape_pages(list_urls[i:i+2000])
    filename = 'carsNov'+str(part)+'.csv'
    part+=1
    data.to_csv(filename, index=False)
    """

18000 20000
10


<ipython-input-71-83eddb1965b0>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Pc4y/Downloads/chromedriver.exe')


1
47
57
72
83
85
86
87
88
89
90
109
124
140
151
179
181
194
195
214
216
228
229
236
242
256
262
269
308
367
376
404
441
442
443
444
445
446
462
488
514
528
542
543
544
545
550
593
646
692
693
698
718
747
757
761
766
787
812
825
826
830
836
854
866
882
884
897
919
938
952
954
966
989
996
1040
1046
1094
1113
1126
1141
1142
1189
1211
1218
1239
1267
1286
1287
1288
1289
1290
1291
1292
1313
1315
1329
1340
1358
1382
1384
1405
1413
1424
1438
1459
1525
1578
1622
1627
1629
1630
1633
1637
1638
1679
1707
1727
1745
1783
1792
1798
1799
1827
1830
1863
1996
2000
20000 22000
11
24
39
53
72
82
97
99
110
189
233
259
270
276
306
332
355
370
399
406
409
416
425
438
443
457
458
465
468
469
491
493
501
505
528
545
546
556
557
559
567
568
585
597
609
610
612
614
615
620
630
667
672
674
681
688
693
700
704
705
760
761
763
781
782
783
805
810
873
921
957
991
997
1018
1045
1047
1048
1052
1059
1060
1068
1079
1105
1111
1130
1138
1143
1181
1191
1195
1252
1266
1274
1279
1303
1309
1312
1321
1327
1341
1369
1371
1374
1